In [ ]:
# ===== DB Connection =====
import pandas as pd
import io
from sqlalchemy.engine import url as sa_url
from sqlalchemy import create_engine
from connection import connectionDB

db_connect_url = sa_url.URL(
            drivername='postgresql+psycopg2',
            username=connectionDB.config.get('username'),
            password=connectionDB.config.get('password'),
            host=connectionDB.config.get('host'),
            port=str(connectionDB.config.get('port')),
            database=connectionDB.config.get('database'),
        )

engine= create_engine(
            db_connect_url
            )

# ===== show your tables from database =====
engine.table_names()

In [ ]:
# ====== Reading table ======
# Reading PostgreSQL table into a pandas DataFrame
def read_sql(query, db_engine):
    copy_sql = "COPY ({query}) TO STDOUT WITH CSV {head}".format(
                    query=query, head="HEADER"
                  )
    conn = db_engine.raw_connection()
    cur = conn.cursor()
    store = io.StringIO()
    cur.copy_expert(copy_sql, 
                    store
                    )
    store.seek(0)
    df = pd.read_csv(store)
    return df

In [ ]:
# ===== Create your complex query =====

query = """
with 
	CRE as (
	select 
		"Date received",
		"Product",
		"Complaint ID",
		"Client_ID",
		"Submitted via"
	from "CRMEvents"
),
	CRL as (
	select
		"Date received" as date_log,
		"Complaint ID",
		"type",
		"outcome",
		ser_time
	from "CRMCallCenterLogs"
),
	combine_df as(
	select 
		CRE."Complaint ID",
		date_log,
		"Product",
		"Submitted via",
        "Client_ID",
		"type",
		"outcome",
		ser_time
	from CRE 
	join CRL on CRE."Complaint ID" = CRL."Complaint ID" and 
                CRE."Date received" = CRL.date_log
),
	prepare_aggregation as (
		select
		to_char(avg(ser_time::time) 
                    over(partition by "Product"),
                        'HH24:MI:SS') as "Average_time_to_resolve_complaints",
		ser_time,
		"Product",
		Class_time
		from(
			select 
			    ser_time,
			    "Product",
			    CASE
			      WHEN (ser_time <= '00:02:00') THEN 'up to 2 minute'
			      WHEN (ser_time between '00:02:00' and '00:05:00') THEN '2 - 5 minute'
			      WHEN (ser_time between '00:05:00' and '00:08:00') THEN '5 - 8 minute'
			      WHEN (ser_time between '00:08:00' and '00:10:00') THEN '8 - 10 minute'
			      WHEN (ser_time > '00:10:00') THEN '10+ minute'
			      ELSE '00'
			    END AS Class_time
			from combine_df as d
			where ser_time is not null
			)h
		group by ser_time,2,3,4
),
    final_aggregation as (
    select
        *,
        LAST_VALUE (cum_amt) over (partition by "Product" 
                                   order by cum_amt 
                                   range between 
                                            unbounded preceding and 
                                            unbounded following ) as cumulative_number_of_complaints,
        round((sum (total)/LAST_VALUE (cum_amt) over (partition by "Product" 
                                                      order by cum_amt 
                                                      range between 
                                                                unbounded preceding and 
                                                                unbounded following ))*100, 2) as percentage
    from(
        select
             *,
            sum(total) over (partition by "Product" 
                             order by "Product" asc 
                             rows between unbounded preceding and current row) as cum_amt
        from(
            select
                "Average_time_to_resolve_complaints",
                "Product",
                Class_time,
                count(Class_time)total
            from  prepare_aggregation
            group by 1,2,3)k
        )l
    group by 1,2,3,4, cum_amt
    order by "Product" asc
)
            
-- final query
select 
    "Average_time_to_resolve_complaints",
    "Product",
    class_time,
    total as Cumulative_complaints,
    percentage
from final_aggregation fl
order by 1 desc, 4 asc
"""

# ===== Running query =====
Summaries = read_sql(query, engine)

## Completed Code

In [ ]:
# ===== DB Connection =====
import pandas as pd
import io
from sqlalchemy.engine import url as sa_url
from sqlalchemy import create_engine
from connection import connectionDB

db_connect_url = sa_url.URL(
            drivername='postgresql+psycopg2',
            username=connectionDB.config.get('username'),
            password=connectionDB.config.get('password'),
            host=connectionDB.config.get('host'),
            port=str(connectionDB.config.get('port')),
            database=connectionDB.config.get('database'),
        )

engine= create_engine(
            db_connect_url
            )

# ====== Reading table ======
# Reading PostgreSQL table into a pandas DataFrame
def read_sql(query, db_engine):
    copy_sql = "COPY ({query}) TO STDOUT WITH CSV {head}".format(
                    query=query, head="HEADER"
                  )
    conn = db_engine.raw_connection()
    cur = conn.cursor()
    store = io.StringIO()
    cur.copy_expert(copy_sql, 
                    store
                    )
    store.seek(0)
    df = pd.read_csv(store)
    return df

# ===== Create your complex query =====
query = """
with 
	CRE as (
	select 
		"Date received",
		"Product",
		"Complaint ID",
		"Client_ID",
		"Submitted via"
	from "CRMEvents"
),
	CRL as (
	select
		"Date received" as date_log,
		"Complaint ID",
		"type",
		"outcome",
		ser_time
	from "CRMCallCenterLogs"
),
	combine_df as(
	select 
		CRE."Complaint ID",
		date_log,
		"Product",
		"Submitted via",
        "Client_ID",
		"type",
		"outcome",
		ser_time
	from CRE 
	join CRL on CRE."Complaint ID" = CRL."Complaint ID" and 
                CRE."Date received" = CRL.date_log
),
	prepare_aggregation as (
		select
		to_char(avg(ser_time::time) 
                    over(partition by "Product"),
                        'HH24:MI:SS') as "Average_time_to_resolve_complaints",
		ser_time,
		"Product",
		Class_time
		from(
			select 
			    ser_time,
			    "Product",
			    CASE
			      WHEN (ser_time <= '00:02:00') THEN 'up to 2 minute'
			      WHEN (ser_time between '00:02:00' and '00:05:00') THEN '2 - 5 minute'
			      WHEN (ser_time between '00:05:00' and '00:08:00') THEN '5 - 8 minute'
			      WHEN (ser_time between '00:08:00' and '00:10:00') THEN '8 - 10 minute'
			      WHEN (ser_time > '00:10:00') THEN '10+ minute'
			      ELSE '00'
			    END AS Class_time
			from combine_df as d
			where ser_time is not null
			)h
		group by ser_time,2,3,4
),
    final_aggregation as (
    select
        *,
        LAST_VALUE (cum_amt) over (partition by "Product" 
                                   order by cum_amt 
                                   range between 
                                            unbounded preceding and 
                                            unbounded following ) as cumulative_number_of_complaints,
        round((sum (total)/LAST_VALUE (cum_amt) over (partition by "Product" 
                                                      order by cum_amt 
                                                      range between 
                                                                unbounded preceding and 
                                                                unbounded following ))*100, 2) as percentage
    from(
        select
             *,
            sum(total) over (partition by "Product" 
                             order by "Product" asc 
                             rows between unbounded preceding and current row) as cum_amt
        from(
            select
                "Average_time_to_resolve_complaints",
                "Product",
                Class_time,
                count(Class_time)total
            from  prepare_aggregation
            group by 1,2,3)k
        )l
    group by 1,2,3,4, cum_amt
    order by "Product" asc
)
            
-- final query
select 
    "Average_time_to_resolve_complaints",
    "Product",
    class_time,
    total as Cumulative_complaints,
    percentage
from final_aggregation fl
order by 1 desc, 4 asc
"""

def main():
    df=read_sql(query, engine)
    return display (df)

if __name__ == "__main__":
    main()